### Домашнее задание <a class="anchor" id="hw"></a><center>

#### 1. Подберите скорость обучения (eta) и количество итераций

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [79]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

def mserror(X, w, y):
    y_pred = X.dot(w)
    return (np.sum((y_pred - y)**2)) / len(y)

In [80]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])

y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [104]:
n = X.shape[0]

#------------------------------------------------------------------------------------
# Было
# eta = 1e-2 
# n_iter = 100

#------------------------------------------------------------------------------------
# Стало - Подобраннные значения:
eta = 1e-2
n_iter = 10000
#------------------------------------------------------------------------------------
# После 8000 - 8500 итераций коэффициенты и MSE не меняются до шестого знака
#------------------------------------------------------------------------------------

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    
    if i % 200 == 0:
#------------------------------------------------------------------------------------
# Было
        eta /= 1.1
#------------------------------------------------------------------------------------
# Стало
# снизил темп уменьшения скорости обучения
        eta /= 1.05
#------------------------------------------------------------------------------------

        print(f'Iteration #{i}: W_new = {W}; eta = {round(eta, 12):12}; MSE = {round(err, 8)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27]; eta = 0.008658008658; MSE = 3047.75
Iteration #200: W_new = [36.52137653  5.36130105]; eta = 0.007496111392; MSE = 78.87697085
Iteration #400: W_new = [42.8964223   4.20528479]; eta = 0.006490139734; MSE = 46.20911384
Iteration #600: W_new = [44.40175747  3.93231547]; eta = 0.0056191686; MSE = 44.17683128
Iteration #800: W_new = [44.82603479  3.85537932]; eta = 0.004865081039; MSE = 43.99535782
Iteration #1000: W_new = [44.96533183  3.83011995]; eta = 0.004212191376; MSE = 43.97323665
Iteration #1200: W_new = [45.01749983  3.82066009]; eta = 0.00364691894; MSE = 43.96971113
Iteration #1400: W_new = [45.0393877   3.81669106]; eta = 0.003157505576; MSE = 43.96900327
Iteration #1600: W_new = [45.0495174   3.81485419]; eta = 0.002733771061; MSE = 43.96882984
Iteration #1800: W_new = [45.05461981  3.81392895]; eta = 0.002366901352; MSE = 43.96877939
Iteration #2000: W

#### 2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [98]:
n = X.shape[0]

eta = 1e-2 
n_iter = 100

W = np.array([1, 0.5])

print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)

#------------------------------------------------------------------------------------
# Было
#     W -= eta * (1/n * 2 * np.dot(X, y_pred - y))
#------------------------------------------------------------------------------------
# Стало
    W -= eta * (2/n * np.dot(X.T, y_pred - y))
#------------------------------------------------------------------------------------

    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #10: W_new = [ 7.0011236 10.6169007], MSE = 738.65
Iteration #20: W_new = [10.3486292  10.10603105], MSE = 622.03
Iteration #30: W_new = [13.38789582  9.55618391], MSE = 525.24
Iteration #40: W_new = [16.16088505  9.05336203], MSE = 444.66
Iteration #50: W_new = [18.69110735  8.59454545], MSE = 377.58
Iteration #60: W_new = [20.99981865  8.17589626], MSE = 321.72
Iteration #70: W_new = [23.10641138  7.79389815], MSE = 275.22
Iteration #80: W_new = [25.02858024  7.44534246], MSE = 236.5
Iteration #90: W_new = [26.78247081  7.12730145], MSE = 204.27


#### 3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога $\epsilon$.

In [103]:
n = X.shape[0]

eta = 1e-2 
n_iter = 100

W = np.array([1, 0.5])

print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

#------------------------------------------------------------------------------------
# Новый код
# Последующий фрагмент кода почти полностью переписан

W_prev = np.zeros(X.shape[1])     # начальные коэффициенты
min_weight_dist = 1e-8            # критерий сходимости
weight_dist = np.inf              # начальная разница весов
i = 1

while weight_dist > min_weight_dist:
    y_pred = np.dot(X, W)

    W -= eta * (2/n * np.dot(X.T, y_pred - y))
    weight_dist = np.linalg.norm(W - W_prev, ord = 2)     # L2-норма
    err = mserror(X, W, y)

    if i % 50 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 8)}; w_dist = {weight_dist:12}')

    W_prev = np.copy(W)
    i += 1
#------------------------------------------------------------------------------------

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #50: W_new = [18.44838865  8.63855875], MSE = 377.57641517; w_dist = 0.24894737331124467
Iteration #100: W_new = [28.22929764  6.86494171], MSE = 177.42704442; w_dist = 0.1574571270703258
Iteration #150: W_new = [34.41564059  5.7431438 ], MSE = 97.3581598; w_dist = 0.09959031314720329
Iteration #200: W_new = [38.32845066  5.03361602], MSE = 65.32695101; w_dist = 0.06299003835074193
Iteration #250: W_new = [40.80327023  4.58484565], MSE = 52.51300535; w_dist = 0.039840671306690605
Iteration #300: W_new = [42.36857287  4.30100215], MSE = 47.38684217; w_dist = 0.02519889067743907
Iteration #350: W_new = [43.35861367  4.12147351], MSE = 45.33614289; w_dist = 0.01593808714931448
Iteration #400: W_new = [43.98480618  4.00792316], MSE = 44.51576958; w_dist = 0.010080706537082105
Iteration #450: W_new = [44.38086769  3.93610351], MSE = 44.1875828; w_dist = 0.006375962393396116
Iteration #500: W_